In [ ]:
%cd /content
!pip install torchsde einops diffusers transformers accelerate peft timm kornia aiohttp
!apt install -qqy

!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI
!git clone https://github.com/ltdrdata/ComfyUI-Manager /content/ComfyUI/custom_nodes/ComfyUI-Manager

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/flux1-fill-dev.safetensors -d /content/ComfyUI/models/unet -o flux1-fill-dev.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/clip_l.safetensors -d /content/ComfyUI/models/clip -o clip_l.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/t5xxl_fp16.safetensors -d /content/ComfyUI/models/clip -o t5xxl_fp16.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/FLUX.1-dev/resolve/main/ae.sft -d /content/ComfyUI/models/vae -o ae.sft

In [ ]:
%cd /content/ComfyUI

import os, shutil, json, requests, random, time
from urllib.parse import urlsplit

import torch
from PIL import Image
import numpy as np

from nodes import NODE_CLASS_MAPPINGS
from comfy_extras import  nodes_flux, nodes_differential_diffusion

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
DifferentialDiffusion = nodes_differential_diffusion.NODE_CLASS_MAPPINGS["DifferentialDiffusion"]()
DualCLIPLoader = NODE_CLASS_MAPPINGS["DualCLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
LoadImage = NODE_CLASS_MAPPINGS["LoadImage"]()

ImagePadForOutpaint =  NODE_CLASS_MAPPINGS["ImagePadForOutpaint"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
FluxGuidance = nodes_flux.NODE_CLASS_MAPPINGS["FluxGuidance"]()
InpaintModelConditioning = NODE_CLASS_MAPPINGS["InpaintModelConditioning"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("flux1-fill-dev.safetensors", "default")[0]
    unet = DifferentialDiffusion.apply(unet)[0]
    clip = DualCLIPLoader.load_clip("google_t5-v1_1-xxl_encoderonly-fp8_e4m3fn.safetensors", "clip_l.safetensors", "flux")[0]
    vae = VAELoader.load_vae("ae.sft")[0]

def download_file(url, save_dir, file_name):
    os.makedirs(save_dir, exist_ok=True)
    file_suffix = os.path.splitext(urlsplit(url).path)[1]
    file_name_with_suffix = file_name + file_suffix
    file_path = os.path.join(save_dir, file_name_with_suffix)
    response = requests.get(url)
    response.raise_for_status()
    with open(file_path, 'wb') as file:
        file.write(response.content)
    return file_path

@torch.inference_mode()
def generate(input):
    values = input["input"]

    input_image = values['input_image']
    input_image = download_file(url=input_image, save_dir='/content/ComfyUI/input', file_name='input_image')
    left = values['left']
    right = values['right']
    top = values['top']
    bottom = values['bottom']
    feathering = values['feathering']
    positive_prompt = values['positive_prompt']
    negative_prompt = values['negative_prompt']
    seed = values['seed']
    steps = values['steps']
    guidance = values['guidance']
    cfg = values['cfg']
    sampler_name = values['sampler_name']
    scheduler = values['scheduler']

    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)
    print(seed)

    conditioning_positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    conditioning_negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    image = LoadImage.load_image(input_image)[0]
    pixels, mask = ImagePadForOutpaint.expand_image(image, left, top, right, bottom, feathering=feathering)
    conditioning_positive = FluxGuidance.append(conditioning_positive, guidance)[0]
    positive, negative, latent_image = InpaintModelConditioning.encode(conditioning_positive, conditioning_negative, pixels, vae, mask, noise_mask=False)
    samples = KSampler.sample(unet, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent_image, denoise=1.0)[0]
    decoded = VAEDecode.decode(vae, samples)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(f"/content/outpaint-flux-fill-{seed}-tost.png")

    result = f"/content/outpaint-flux-fill-{seed}-tost.png"

    return result

In [ ]:
input = { 
        "input": {
        "input_image": "https://files.catbox.moe/pqi6yz.png",
        "left": 600,
        "right": 600,
        "top": 0,
        "bottom": 0,
        "feathering": 24,
        "positive_prompt": "",
        "negative_prompt": "",
        "seed": 0,
        "steps": 20,
        "guidance": 30,
        "cfg": 1.0,
        "sampler_name": "euler",
        "scheduler": "normal"
    }
}
image = generate(input)
Image.open(image)